In [ ]:
NIFTY_50 = [
    "RELIANCE", "HDFCBANK", "TCS", "INFY", "ICICIBANK", "ITC", "LT", "HINDUNILVR",
    "KOTAKBANK", "SBIN", "BAJFINANCE", "BHARTIARTL", "ASIANPAINT", "HCLTECH",
    "MARUTI", "AXISBANK", "SUNPHARMA", "TITAN", "ULTRACEMCO", "INDUSINDBK",
    "ONGC", "M&M", "NTPC", "POWERGRID", "TATAMOTORS", "BPCL", "ADANIPORTS",
    "DIVISLAB", "BAJAJFINSV", "JSWSTEEL", "EICHERMOT", "COALINDIA", "GRASIM",
    "BRITANNIA", "CIPLA", "HDFCLIFE", "HEROMOTOCO", "SHREECEM", "UPL",
    "WIPRO", "SBILIFE", "DRREDDY", "TATACONSUM", "HDFC", "BAJAJ-AUTO",
    "APOLLOHOSP", "ADANIENT", "TATASTEEL", "SALE", "NESTLEIND", "PIDILITIND"
]


In [ ]:
import time
from datetime import datetime, time as dtime
import pandas as pd
from groww_trade import Groww
import talib

gw = Groww("YOUR_GROWW_API_KEY", "YOUR_GROWW_API_SECRET")
EXCHANGE = "NSE"

def is_market_open():
    now = datetime.now().time()
    return dtime(10, 0) <= now <= dtime(15, 0)

def square_off_all():
    positions = gw.get_positions()
    for pos in positions:
        if pos['exchange'] == EXCHANGE and pos['product'] == 'INTRADAY':
            side = 'SELL' if pos['side'] == 'BUY' else 'BUY'
            gw.place_order(side, pos['symbol'], quantity=pos['quantity'], order_type='MARKET')
            print(f"Squared off {pos['symbol']} at day close.")

def run_strategy():
    for ticker in NIFTY_50:
        # Fetch historical data
        try:
            data = gw.get_historical_ohlc(ticker, interval='1d', exchange=EXCHANGE,
                                          range_from='2025-07-01', range_to=datetime.now().strftime('%Y-%m-%d'))
            df = pd.DataFrame(data)
            upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20, nbdevup=2, nbdevdn=2)
            df['bb_upper'], df['bb_lower'] = upper, lower
            
            # Signal logic
            latest = df.iloc[-1]
            if latest['close'] < latest['bb_lower']:
                gw.place_order('BUY', ticker, quantity=1, order_type='MARKET', product='INTRADAY')
            elif latest['close'] > latest['bb_upper']:
                gw.place_order('SELL', ticker, quantity=1, order_type='MARKET', product='INTRADAY')
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

if __name__ == "__main__":
    already_squared_off = False
    while True:
        if is_market_open():
            run_strategy()
            time.sleep(300)  # adjust frequency as needed (e.g., every 5 min)
            already_squared_off = False
        else:
            # Square off positions at or after 3PM (market close)
            if not already_squared_off and datetime.now().time() >= dtime(15, 0):
                square_off_all()
                already_squared_off = True
            time.sleep(60)
